<a href="https://colab.research.google.com/github/AlvingEj/monitoreo_control_arvenses/blob/main/Entrenamiento_de_la_Red_Neuronal_Convolucional_para_Clasificaci%C3%B3n_de_Im%C3%A1genes_Proceso_y_Resultados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Clasificación de Hojas de Frutas con Redes Neuronales Convolucionales y Generación de Informe PDF"**
---







In [ ]:
# Instalar paquetes necesarios

#Permite la generación de documentos PDF de manera flexible.
!pip install reportlab
#proporciona información sobre zonas horarias y operaciones con fechas y horas.
!pip install pytz

In [ ]:
# Manejo de archivos y sistema operativo
import os

# Manejo de archivos zip
import zipfile

# Subida y manejo de archivos en Google Colab
from google.colab import files

# Manejo de imágenes
from PIL import Image

# Manejo de tiempo y zona horaria
from datetime import datetime
import pytz

# Librerías para creación de gráficos
import matplotlib.pyplot as plt
import numpy as np

# TensorFlow y Keras para la creación y entrenamiento de modelos de redes neuronales
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Librerías para la generación de PDFs
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

In [ ]:
# Función para subir y descomprimir archivo zip
def subir_y_descomprimir_zip():
    # Subir archivo ZIP desde el entorno de Google Colab
    uploaded = files.upload()

    if uploaded:
        # Obtener el nombre del archivo ZIP subido
        nombre_archivo_zip = list(uploaded.keys())[0]

        try:
            # Descomprimir el archivo ZIP en el directorio 'dataset'
            with zipfile.ZipFile(nombre_archivo_zip, 'r') as zip_ref:
                zip_ref.extractall('dataset')
            print("Archivo zip descomprimido correctamente")
        except zipfile.BadZipFile:
            # Manejar el error si el archivo no es un ZIP válido
            print("Error: El archivo subido no es un archivo zip válido.")
    else:
        # Mensaje de error si no se subió ningún archivo
        print("Error: No se subió ningún archivo zip.")

# Llamada a la función para subir y descomprimir el archivo
subir_y_descomprimir_zip()

In [ ]:
# Función para verificar la estructura de directorios
def verificar_estructura_directorios():
    print("Contenido de la carpeta 'dataset':")
    print(os.listdir('dataset'))

    print("Contenido de la carpeta 'dataset/dataset/Manzana_Sana':")
    print(os.listdir('dataset/dataset/Manzana_Sana'))
    print("Contenido de la carpeta 'dataset/dataset/Uva_Sana':")
    print(os.listdir('dataset/dataset/Uva_Sana'))

# Llamada a la función para verificar la estructura de directorios
verificar_estructura_directorios()

In [ ]:
# Configurar generadores de imágenes
def configurar_generadores_imagenes():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    generador_entrenamiento = datagen.flow_from_directory(
        'dataset/dataset',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )

    generador_validacion = datagen.flow_from_directory(
        'dataset/dataset',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    print(f"Se encontraron {generador_entrenamiento.samples} imágenes pertenecientes a {generador_entrenamiento.num_classes} clases para entrenamiento.")
    print(f"Se encontraron {generador_validacion.samples} imágenes pertenecientes a {generador_validacion.num_classes} clases para validación.")

    return generador_entrenamiento, generador_validacion

# Llamada a la función para configurar los generadores de imágenes
generador_entrenamiento, generador_validacion = configurar_generadores_imagenes()


In [ ]:
# Construir y compilar el modelo
def construir_y_compilar_modelo():
    modelo = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    modelo.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

    modelo.summary()
    return modelo

# Llamada a la función para construir y compilar el modelo
modelo = construir_y_compilar_modelo()

In [ ]:
# Entrenar el modelo
historial = modelo.fit(
    generador_entrenamiento,
    epochs=25,
    validation_data=generador_validacion
)


In [ ]:
# Graficar métricas de entrenamiento
def graficar_metricas_entrenamiento(historial):
    acc = historial.history['accuracy']
    val_acc = historial.history['val_accuracy']
    loss = historial.history['loss']
    val_loss = historial.history['val_loss']

    epocas = range(len(acc))

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epocas, acc, 'r', label='Precisión de entrenamiento')
    plt.plot(epocas, val_acc, 'b', label='Precisión de validación')
    plt.title('Precisión de entrenamiento y validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epocas, loss, 'r', label='Pérdida de entrenamiento')
    plt.plot(epocas, val_loss, 'b', label='Pérdida de validación')
    plt.title('Pérdida de entrenamiento y validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.show()

graficar_metricas_entrenamiento(historial)

In [ ]:
# Función para predecir y mostrar imágenes, además de guardar la imagen procesada y la predicción
def predecir_guardar_y_mostrar_imagen(ruta_imagen, modelo, directorio_salida):
    try:
        img = Image.open(ruta_imagen)
        img_redimensionada = img.resize((150, 150))
        img_array = tf.keras.preprocessing.image.img_to_array(img_redimensionada)
        img_array = tf.expand_dims(img_array, axis=0) / 255.0

        prediccion = modelo.predict(img_array)
        etiqueta_clase = "Hoja de Uva" if prediccion[0] > 0.5 else "Hoja de Manzana"

        print(f"{os.path.basename(ruta_imagen)}: Predicción - {etiqueta_clase}")

        ruta_guardar_imagen = f"{directorio_salida}/{os.path.basename(ruta_imagen)}"
        img_redimensionada.save(ruta_guardar_imagen)

        plt.figure(figsize=(6, 6))
        plt.imshow(img_redimensionada)
        plt.title(f"Predicción: {etiqueta_clase}")
        plt.axis('off')
        plt.show()

        return ruta_guardar_imagen, etiqueta_clase

    except Exception as e:
        print(f"Error al procesar {ruta_imagen}: {e}")
        return ruta_imagen, "Error"

In [ ]:
# Cargar imágenes y predecir
subidos = files.upload()
directorio_salida = 'imagenes_procesadas'
os.makedirs(directorio_salida, exist_ok=True)

resultados = []

for nombre_archivo in subidos.keys():
    ruta_imagen = nombre_archivo
    ruta_guardar_imagen, prediccion = predecir_guardar_y_mostrar_imagen(ruta_imagen, modelo, directorio_salida)
    resultados.append((ruta_guardar_imagen, prediccion))

In [ ]:
# Generar informe en PDF
def generar_informe_pdf(resultados, ruta_pdf='Informe_Predicciones.pdf'):
    c = canvas.Canvas(ruta_pdf, pagesize=letter)
    ancho, alto = letter

    margen = 50
    ancho_imagen = 2 * inch
    alto_imagen = 2 * inch
    altura_linea = 20

    zona_horaria_local = pytz.timezone('America/Bogota')
    hora_local = datetime.now(zona_horaria_local).strftime('%d-%m-%y %H:%M:%S')

    c.setFont("Helvetica-Bold", 12)
    c.drawString(margen, alto - margen + 20, "Informe de Predicciones")
    c.setFont("Helvetica", 10)
    c.drawString(margen, alto - margen, f"Fecha y Hora: {hora_local}")

    posicion_y = alto - margen - 40

    for ruta_guardar_imagen, prediccion in resultados:
        if posicion_y - (alto_imagen + altura_linea) < margen:
            c.showPage()
            c.setFont("Helvetica-Bold", 12)
            c.drawString(margen, alto - margen + 20, "Informe de Predicciones")
            c.setFont("Helvetica", 10)
            c.drawString(margen, alto - margen, f"Fecha y Hora: {hora_local}")
            posicion_y = alto - margen - 40

        c.drawString(margen, posicion_y, f"{os.path.basename(ruta_guardar_imagen)}: Predicción - {prediccion}")
        posicion_y -= altura_linea

        c.drawImage(ruta_guardar_imagen, margen, posicion_y - alto_imagen, width=ancho_imagen, height=alto_imagen)
        posicion_y -= alto_imagen + altura_linea

    c.save()

    print("Informe PDF generado como 'Informe_Predicciones.pdf'")
    files.download(ruta_pdf)

generar_informe_pdf(resultados)

In [ ]:
###no borrar - imagen ecualizada

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Cargar la imagen en escala de grises
# Usa la opción de Google Colab para subir archivos o carga una imagen desde una URL

from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    image_path = filename

# Leer la imagen en escala de grises
imagen_original = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Aplicar ecualización de histograma
imagen_ecualizada = cv2.equalizeHist(imagen_original)

# Mostrar imagen original y ecualizada usando cv2_imshow
print("Imagen Original")
cv2_imshow(imagen_original)

print("Imagen Ecualizada")
cv2_imshow(imagen_ecualizada)

# Guardar la imagen ecualizada
cv2.imwrite('imagen_ecualizada.jpg', imagen_ecualizada)
